In [1]:
import tensorflow as tf
import chess.pgn
import pandas as pd
import numpy as np
import os
import time
import multiprocessing as mp
import random
import io
import re
import dask.dataframe as dd
import glob
    


2025-05-28 22:43:18.131435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748493798.151802   10676 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748493798.157440   10676 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-28 22:43:18.181800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

try:
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print('Now using gpu')
    else: print('Unable to use the gpu')


except RuntimeError as e:
    print(f'Something went wrong: {e}')


Now using gpu


In [3]:
file = '../data/raw/lichess_db_standard_rated_2016-03.pgn'

In [4]:
# def game_pipeline_from_text(game_text):
#     game = chess.pgn.read_game(io.StringIO(game_text))
#     if game is None:
#         return None
#
#     moves = list(game.mainline_moves())
#
#     game_list = []
#     board = game.board()
#     result = game.headers['Result']
#     outcome = 1 if result == '1-0' else -1 if result == '0-1' else 0
#
#     for move in moves:
#         fen = board.fen()
#         move_uci = move.uci()
#         move_san = board.san(move)
#         player = 'white' if board.turn == chess.WHITE else 'black'
#         game_list.append({
#             'fen': fen,
#             'move_uci': move_uci,
#             'move_san': move_san,
#             'player': player,
#             'result': outcome
#         })
#         board.push(move)
#
#     return game_list if game_list else None
#
# def process_and_save_chunk(games_chunk, chunk_idx):
#     results = list(filter(None, map(lambda g: game_pipeline_from_text(g), games_chunk)))
#     flat_results = [entry for game in results for entry in game]  # Flatten list
#     df = pd.DataFrame(flat_results)
#     df.to_csv(f"../data/csv/processed_chunk_{chunk_idx}.csv", index=False)
#     print(f"Saved chunk {chunk_idx} with {len(results)} valid games")
#
#
# def get_completed_chunk_ids(path='../data/csv/'):
#
#     completed_chunks = os.listdir(path)
#     completed_chunks_ids = [int(re.search(r'\d+', chunk)[0]) for chunk in completed_chunks]
#     return completed_chunks_ids
#
# chunk_size = 10_000
# game_idx = 0
# games_chunk = []
# completed_chunks_ids = get_completed_chunk_ids()
#
# with open(file) as f:
#     while True:
#         game = chess.pgn.read_game(f)
#         if game is None:
#             break
#
#         current_chunk_idx = game_idx // chunk_size
#
#         if current_chunk_idx in completed_chunks_ids:
#             game_idx +=1
#             continue
#         games_chunk.append(str(game))
#         game_idx += 1
#
#         if len(games_chunk) >= chunk_size:
#             print(f'Processing chunk {current_chunk_idx} ')
#             process_and_save_chunk(games_chunk, current_chunk_idx)  # usa un número razonable o calcula primero
#             games_chunk = []
#
#
# # Procesar último chunk
# if games_chunk:
#     current_chunk_idx = game_idx // chunk_size
#     print(f"Processing final chunk {current_chunk_idx}...")
#     process_and_save_chunk(games_chunk, current_chunk_idx)


In [5]:
#helper function to assing id to the chunks

def add_game_id_col(df_chunk):
    df_chunk['game_id'] = pd.Series()
    idx_start_game = df_chunk.loc[df_chunk['fen']=='rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'].index.to_list()
    game_id = 0
    for idx in idx_start_game:
        df_chunk.iloc[idx,-1] = game_id #iterate over each start of game and add the value in the last created column
        game_id += 1
    df_chunk['game_id'] = df_chunk['game_id'].ffill()
    return df_chunk




In [6]:
chunks_dir = '../data/csv/*.csv'


In [9]:
parquet_dir = '../data/processed/'

In [7]:
one_million_records = pd.concat([pd.read_csv(chunk) for chunk in glob.glob(chunks_dir)]).reset_index(drop=True)

In [8]:
one_million_records = add_game_id_col(one_million_records)

/tmp/ipykernel_10676/2475813830.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_chunk['game_id'] = df_chunk['game_id'].ffill()


In [17]:

chunk_size = 10_000_000  # por ejemplo 10 millones filas por archivo
num_chunks = len(one_million_records) // chunk_size + 1

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i+1)*chunk_size, len(one_million_records))
    df_chunk = one_million_records.iloc[start:end]
    df_chunk.to_parquet(os.path.join(parquet_dir,f'parquet_part_{i}.parquet'), index=False)


In [21]:
one_million_records.to_parquet(os.path.join(parquet_dir,f'One_million_games.parquet'), index=False)